In [12]:
import pandas as pd

obj = {}

df = pd.read_csv('./tweets.csv')
obj['text'] = df['text']
dataframe = pd.DataFrame(obj)
dataframe.head(10)

/home/buckaroo/nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (27,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,text
0,RT @KirkKus: Indirect cost of the UK being in ...
1,VIDEO: Sturgeon on post-election deals http://...
2,RT @LabourEoin: The economy was growing 3 time...
3,RT @GregLauder: the UKIP east lothian candidat...
4,RT @thesundaypeople: UKIP's housing spokesman ...
5,RT @Nigel_Farage: Make sure you tune in to #As...
6,RT @joannetallis: Ed Milliband is an embarrass...
7,RT @abstex: The FT is backing the Tories. On a...
8,RT @NivenJ1: “@George_Osborne: Ed Miliband pro...
9,LOLZ to Trickle Down Wealth. It's never trickl...


In [13]:
print(dataframe.shape)

(20000, 1)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(dataframe['text'])
tfidf = vectorizer.transform(dataframe['text'])
vocab = vectorizer.get_feature_names()
print(vocab[0])
print(len(vocab))

000
9006


In [15]:
from corextopic import corextopic as ct

In [16]:
anchors = []
model = ct.Corex(n_hidden=4, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [17]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: rt nicolasturgeon, nicolasturgeon if, will definitely, we will, definitely need, protect sco, is going, need lots, in rather, snp we
Topic #2: tories claiming, claiming, preoccupied, is preoccupied, in support, preoccupied inequality, inequality the, wrote it, who wrote, claiming miliband
Topic #3: says he, are all, never forgive, says, you hear, off the, forgive, into, off, into new
Topic #4: snp, with, rather, with snp, deal, let, work with, work, deal with, tories in


In [18]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["money","fund"],
    ["emergency"],
    ["recovered"],
    ["treatment"]
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=4, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [19]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: is preoccupied, tommy_colc, preoccupied, in support, who wrote, preoccupied inequality, rt tommy_colc, inequality the, wrote it, claiming miliband
Topic #2: rather, with, let, than work, work with, than, work, let tories, tories in, with snp
Topic #3: says he, global financial, caused global, thus caused, thomasmessenger for, caused, and thus, ahem http, crisis ahem, thomasmessenger
Topic #4: http, http co, co, financial, bbcqt http, david cameron, david, cameron


In [20]:
def arr_to_topic(arr=[]):
    array = ""
    for s in arr:
        array+=s+" "
        break
    
    return array   

In [21]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    #columns=["topic_{}".format(i+1) for i in range(4)]
    columns=  ["money","emergency","recovered","treatment"]
).astype(float)


topic_df.index = dataframe.index

print(dataframe.shape)
print(topic_df.shape)

topic_df

final = pd.concat([dataframe, topic_df], axis=1)

(20000, 1)
(20000, 4)


In [22]:
final.sample(100, random_state=92)


,text,money,emergency,recovered,treatment
15119,RT @STUD6699: Nigel Farage says that homosexua...,0.0,0.0,1.0,0.0
12499,RT @UKIPEaling: Nigel Farage speaking with con...,0.0,0.0,0.0,0.0
10326,@ukmarkus http://t.co/WkfVKwHgED,0.0,0.0,0.0,1.0
238,RT @A_Liberty_Rebel: Only regret at #BBCQT is ...,0.0,0.0,0.0,0.0
1943,#bbcqt I'm surprised Cameron only got 44%. The...,0.0,0.0,0.0,1.0
...,...,...,...,...,...
7762,RT @UKIP: 70% of Britons support an Australian...,0.0,0.0,0.0,1.0
15774,RT @ad5red: @Nigel_Farage Different class toni...,0.0,0.0,0.0,0.0
14655,RT @NicolaSturgeon: If Miliband is going to le...,0.0,1.0,0.0,0.0
16025,Spot on Alison Parr; he did a fine job; unruff...,0.0,0.0,0.0,0.0


In [62]:
! curl -L -o result.json http://ec2-13-232-219-139.ap-south-1.compute.amazonaws.com:4000/api/corona/0/9999

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49894  100 49894    0     0  16252      0  0:00:03  0:00:03 --:--:-- 16252


In [63]:
import json

tweetJSON = {}

with open('./result.json' , encoding='utf-8') as J:
    tweetJSON = json.loads(J.read())
    
tweets = tweetJSON['response']['tweets']

tweet_texts = []

for index,tweet in enumerate(tweets):
    print( index ,". " , tweet['text'] )
    tweet_texts.append(tweet['text'])

0 .  RT @zeitonline: #China | s leadership wants to finally conquer #Hongkong | s democracy in the shadow of #Corona: @agoerlach on the future of the S…
1 .  RT @zarazettirazr: About debt debt debt, blaming corona and impeachment by @RamliRizal
#ILCCrisis Threatening https://t.co/i2UBdw5b…
2 .  Rumors from North Korea that the Great Leader would have Corona.
3 .  RT @MDR_SN: # Corona ticker #Saxony in the evening: further deaths in nursing home in #Radeberg +++ household lock also in #Zwickau https: //t.…
4 .  before corona      vs     while corona https://t.co/n314gavpDl
5 .  RT @Oliseh_King: Corona virus really streaming on all platforms
6 .  The staff of the Käuber Räuberbande in Hemer wrote poems for the parents and children. We have a little outlook ... https://t.co/tnoQDQ92SG
7 .  RT @dani_draco_: Corona be full finesse
8 .  RT @Divine27371914: Hey rap fans, I feel like corona isn’t out there. Kao🇸🇱needs to come again. I analyzed this shit and what LJ did was fl…


In [64]:
vectorizer = TfidfVectorizer(
    max_df=1,
    min_df=1,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(tweet['text'])

tfidf = vectorizer.transform(tweet['text'])

input_text = vectorizer.get_feature_names()

input_text

ValueError: Iterable over raw text documents expected, string object received.

In [52]:
model.predict(input_text)

AttributeError: 'list' object has no attribute 'max'